## Classifying with  _sklearn.neighbors.KNeighborsClassifier_

Using knn to predict some targets from line 144 datasource

### Imports

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

### Loading data

Getting data from 4 years

In [2]:
years = [2020, 2021, 2022, 2023]
data_dict = {}

for year in years:
    data_dict[year] = pd.read_csv(f'./data/linea144-{year}.csv', parse_dates=['fecha'])

data = pd.concat(data_dict, ignore_index=True)

# Excluding last column with no data
data = data.iloc[:,:-1]

assert data.shape == (84861, 19)
assert data.fecha.dtype == 'datetime64[ns]', "'fecha' must be a datetime64 type"

### Cleanning data

* Filling NaN values
* Joining multiple labels to standars

In [3]:
# Filling nan rows with mode
def fill_with_mode_the_nan_values(field):

    # n records NaN to theis field 
    fields_nan = data[field].isna().sum()
    print(f'Processing records NaN to {field}: {fields_nan}')
    
    if fields_nan > 0:
        # filling using the mode, like shown in link above
        field_mode = data[field].mode()[0]

        data[field].fillna(field_mode, inplace=True)

fields = ['prov_residencia_persona_en_situacion_violencia', 'genero_persona_en_situacion_de_violencia', \
          'pais_nacimiento_persona_en_situacion_de_violencia', 'vinculo_con_la_persona_agresora', \
            'genero_de_la_persona_agresora']

[fill_with_mode_the_nan_values(field) for field in fields]

Processing records NaN to prov_residencia_persona_en_situacion_violencia: 1229
Processing records NaN to genero_persona_en_situacion_de_violencia: 1776
Processing records NaN to pais_nacimiento_persona_en_situacion_de_violencia: 28706
Processing records NaN to vinculo_con_la_persona_agresora: 3192
Processing records NaN to genero_de_la_persona_agresora: 8737


[None, None, None, None, None]

In [4]:
# Calculate the median
age_median = data.edad_persona_en_situacion_de_violencia.median()

# Filling data with the mean
data.fillna(age_median, inplace=True)

In [5]:
boolean_fields = ['tipo_de_violencia_fisica', 'tipo_de_violencia_psicologica', 'tipo_de_violencia_sexual', \
                    'tipo_de_violencia_economica_y_patrimonial', 'tipo_de_violencia_simbolica', 'tipo_de_violencia_domestica',
                    'modalidad_de_violencia_institucional', 'modalidad_de_violencia_laboral', \
                    'modalidad_violencia_contra_libertad_reproductiva', 'modalidad_de_violencia_obstetrica', \
                    'modalidad_de_violencia_mediatica', 'modalidad_de_violencia_otras']

sim = 'Si'
nao = 'No'
mapping = {'Si': sim, 'SI': sim, 'No': nao, 'NO': nao}

for field in boolean_fields:
    data[field] = data[field].map(mapping)


### Labeled data

In [6]:
fields_to_encode = data.columns.drop(['fecha', 'edad_persona_en_situacion_de_violencia'])

for field in fields_to_encode:
    le = LabelEncoder()
    le.fit(data[field])
    data[field] = le.transform(data[field])

### Classifying ...

In [19]:
def linear_regression(_target):
    y = data[_target]
    X = data.drop(['fecha', 'edad_persona_en_situacion_de_violencia', _target], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

    print(X.info())
    

    # linear_regression = LinearRegression()
    # linear_regression.fit(X_train, y_train)

    # predicted = linear_regression.predict(X_test)
    # accuracy = accuracy_score(predicted, y_test)

    # print("Accuracy from {} is {}".format(_target, accuracy))


In [20]:
fields = data.columns.drop("fecha", 'edad_persona_en_situacion_de_violencia')
for target in fields:
    linear_regression(target)
    break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84861 entries, 0 to 84860
Data columns (total 16 columns):
 #   Column                                             Non-Null Count  Dtype
---  ------                                             --------------  -----
 0   genero_persona_en_situacion_de_violencia           84861 non-null  int32
 1   pais_nacimiento_persona_en_situacion_de_violencia  84861 non-null  int32
 2   tipo_de_violencia_fisica                           84861 non-null  int32
 3   tipo_de_violencia_psicologica                      84861 non-null  int32
 4   tipo_de_violencia_sexual                           84861 non-null  int32
 5   tipo_de_violencia_economica_y_patrimonial          84861 non-null  int32
 6   tipo_de_violencia_simbolica                        84861 non-null  int32
 7   tipo_de_violencia_domestica                        84861 non-null  int32
 8   modalidad_de_violencia_institucional               84861 non-null  int32
 9   modalidad_de_violencia_labor

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84861 entries, 0 to 84860
Data columns (total 19 columns):
 #   Column                                             Non-Null Count  Dtype         
---  ------                                             --------------  -----         
 0   fecha                                              84861 non-null  datetime64[ns]
 1   prov_residencia_persona_en_situacion_violencia     84861 non-null  int32         
 2   genero_persona_en_situacion_de_violencia           84861 non-null  int32         
 3   edad_persona_en_situacion_de_violencia             84861 non-null  float64       
 4   pais_nacimiento_persona_en_situacion_de_violencia  84861 non-null  int32         
 5   tipo_de_violencia_fisica                           84861 non-null  int32         
 6   tipo_de_violencia_psicologica                      84861 non-null  int32         
 7   tipo_de_violencia_sexual                           84861 non-null  int32         
 8   tipo_de_violenci